In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tf-roberta/vocab-roberta-base.json
/kaggle/input/tf-roberta/pretrained-roberta-base.h5
/kaggle/input/tf-roberta/merges-roberta-base.txt
/kaggle/input/tf-roberta/config-roberta-base.json


In [2]:
!pip install transformers
!pip install tokenizers
############################imports##########################################


import numpy as np
import pandas as pd

import json

import re
import string
from os import listdir
from collections import Counter
import nltk
from nltk.corpus import stopwords
from transformers import *
import tokenizers


nltk.download("stopwords")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [3]:
MAX_LEN = 96
PATH = '../input/tf-roberta/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv').fillna('')
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')
sample_submission = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')

In [4]:
train_encoded=dict({'input_id':[],
                      'attention_mask':[],
                      'token_id':[],
                      'start_token':[],
                      'end_token':[]})

    #single_sentence=single_sentence_raw.copy()
   
for index in range(train.shape[0]):
    input_id=np.ones((MAX_LEN),dtype='int32');attention_mask=np.zeros((MAX_LEN),dtype='int32');token_id=np.zeros((MAX_LEN),dtype='int32')
    start_token=np.zeros((MAX_LEN),dtype='int32');end_token=np.zeros((MAX_LEN),dtype='int32')
   
    text1 = " "+" ".join(train.loc[index,'text'].split())
    text2 = " ".join(train.loc[index,'selected_text'].split())
    #print("text1 : ",text1)
    #print("text2 : ",text2)

    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': 
      chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
    # ID_OFFSETS
    #print(enc)
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    #print("offsets=",offsets)
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
    s_tok = sentiment_id[train.loc[index,'sentiment']]
    input_id[0:len(enc.ids)+5]=[0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[0:len(enc.ids)+5]=1
    if len(toks)>0:
      start_token[toks[0]+1]=1
      end_token[toks[-1]+1]=1
    
    train_encoded['input_id'].append(input_id)
    train_encoded['attention_mask'].append(attention_mask)
    train_encoded['start_token'].append(start_token)
    train_encoded['end_token'].append(end_token)
    train_encoded['token_id'].append(token_id)
    #print(global()[col])
for col in list(train_encoded.keys()):
  train_encoded[col]=np.array(train_encoded[col])


In [5]:
test_encoded=dict({'input_id':[],
                      'attention_mask':[],
                      'token_id':[]})


for index in range(test.shape[0]):
    input_id=np.ones((MAX_LEN),dtype='int32');attention_mask=np.zeros((MAX_LEN),dtype='int32');token_id=np.zeros((MAX_LEN),dtype='int32')
    text1 = " "+" ".join(test.loc[index,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[index,'sentiment']]
    input_id[0:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[0:len(enc.ids)+5] = 1
    test_encoded['input_id'].append(input_id)
    test_encoded['attention_mask'].append(attention_mask)
    test_encoded['token_id'].append(token_id)
    #print(global()[col])
for col in list(test_encoded.keys()):
  test_encoded[col]=np.array(test_encoded[col])

In [6]:
import tensorflow as tf

def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.15)(x[0]) 
    x1 = tf.keras.layers.Conv1D(768,2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(64, 1,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.15)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768,2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(64, 1,padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model
#jaccard

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [7]:
def get_preds(df,preds,index,isTrain=True):
    all=[];sel_text=[]
    for i in index:
        start_tok=np.argmax(preds['start_token'][i])
        end_tok=np.argmax(preds['end_token'][i])
        if(start_tok>end_tok):
            temp_pred = df.loc[i,'text']
        else:
            text1 = " "+" ".join(df.loc[i,'text'].split())
            enc = tokenizer.encode(text1)
            temp_pred = tokenizer.decode(enc.ids[start_tok-1:end_tok])
        if(isTrain):
            all.append(jaccard(temp_pred,df.loc[i,'selected_text']))
        sel_text.append(temp_pred)

    print(np.mean(all))

    return sel_text

In [8]:
import tensorflow as tf

def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
  
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    
    x1_p = tf.keras.layers.Dropout(0.15)(x[0]) 
    x1_p = tf.keras.layers.Conv1D(768,4,padding='same')(x1_p)
    x1_p = tf.keras.layers.LeakyReLU()(x1_p)
    x1_p = tf.keras.layers.Conv1D(512,2,padding='same')(x1_p)
    x1_p = tf.keras.layers.LeakyReLU()(x1_p)
    x1_p = tf.keras.layers.AveragePooling1D(pool_size=2,padding='same',data_format='channels_first')(x1_p)
    x1_p = tf.keras.layers.Dropout(0.1)(x1_p)
    x1_p = tf.keras.layers.Conv1D(256,1,padding='same')(x1_p)
    x1_p = tf.keras.layers.LeakyReLU()(x1_p)
    x1_p = tf.keras.layers.Conv1D(64, 1,padding='same')(x1_p)
    x1_p = tf.keras.layers.Dense(1)(x1_p)
    x1_p = tf.keras.layers.Flatten()(x1_p)
    x1 = tf.keras.layers.Activation('softmax')(x1_p)

    x2_p = tf.keras.layers.Dropout(0.15)(x[0]) 
    x2_p = tf.keras.layers.Conv1D(768,4,padding='same')(x2_p)
    x2_p = tf.keras.layers.LeakyReLU()(x2_p)
    x2_p = tf.keras.layers.Conv1D(512,2,padding='same')(x2_p)
    x2_p = tf.keras.layers.LeakyReLU()(x2_p)
    x2_p = tf.keras.layers.AveragePooling1D(pool_size=2,padding='same',data_format='channels_first')(x2_p)
    x2_p = tf.keras.layers.Dropout(0.1)(x2_p)
    x2_p = tf.keras.layers.Conv1D(256,1,padding='same')(x2_p)
    x2_p = tf.keras.layers.LeakyReLU()(x2_p)
    x2_p = tf.keras.layers.Conv1D(64, 1,padding='same')(x2_p)
    x2_p = tf.keras.layers.Dense(1)(x2_p)
    x2_p = tf.keras.layers.Flatten()(x2_p)
    x2 = tf.keras.layers.Activation('softmax')(x2_p)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [9]:
from sklearn.model_selection import StratifiedKFold

val_pred=dict({'start_token':np.zeros((train_encoded['input_id'].shape[0],MAX_LEN)),
                      'end_token':np.zeros((train_encoded['input_id'].shape[0],MAX_LEN))})
test_pred=dict({'start_token':np.zeros((test_encoded['input_id'].shape[0],MAX_LEN)),
                      'end_token':np.zeros((test_encoded['input_id'].shape[0],MAX_LEN))})
num_splits=5
sss  = StratifiedKFold(n_splits=num_splits,shuffle=True,random_state=777)

for fold,(train_index,val_index) in enumerate(sss.split(train_encoded['input_id'],train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    #K.clear_session()
    model = build_model()
        
    sv = tf.keras.callbacks.ModelCheckpoint('%s-roberta-%i.h5'%(1,fold), monitor='val_loss', verbose=1, save_best_only=True,
           save_weights_only=True, mode='auto', save_freq='epoch')



    X_train,y_train=[train_encoded['input_id'][train_index], train_encoded['attention_mask'][train_index],train_encoded['token_id'][train_index]], [train_encoded['start_token'][train_index], train_encoded['end_token'][train_index]]
    X_val,y_val=[train_encoded['input_id'][val_index],train_encoded['attention_mask'][val_index],train_encoded['token_id'][val_index]], [train_encoded['start_token'][val_index], train_encoded['end_token'][val_index]]
    test_data=[test_encoded['input_id'],test_encoded['attention_mask'],test_encoded['token_id']]

    model.fit(X_train,y_train ,epochs=3, batch_size=32, verbose=1, callbacks=[sv], validation_data=(X_val,y_val))
    
    model.load_weights('%s-roberta-%i.h5'%(1,fold))
    
    print('Predicting OOF...')
    val_pred['start_token'][val_index,],val_pred['end_token'][val_index,] = model.predict(X_val,verbose=1)
    sel_text_val=get_preds(train,val_pred,val_index,True)
    
    preds = model.predict(test_data,verbose=1)
    test_pred['start_token']+=preds[0]/num_splits
    test_pred['end_token']+=preds[1]/num_splits
    
sel_text_test=get_preds(test,test_pred,np.arange(0,test.shape[0]),False)

#########################
### FOLD 1
#########################
Train on 21984 samples, validate on 5497 samples
Epoch 1/3
21952/21984 [============================>.] - ETA: 0s - loss: 2.0498 - activation_7_loss: 1.0621 - activation_8_loss: 0.9877
Epoch 00001: val_loss improved from inf to 1.73662, saving model to 1-roberta-0.h5
21984/21984 [==============================] - 308s 14ms/sample - loss: 2.0488 - activation_7_loss: 1.0615 - activation_8_loss: 0.9873 - val_loss: 1.7366 - val_activation_7_loss: 0.8956 - val_activation_8_loss: 0.8407
Epoch 2/3
21952/21984 [============================>.] - ETA: 0s - loss: 1.5938 - activation_7_loss: 0.8210 - activation_8_loss: 0.7728
Epoch 00002: val_loss improved from 1.73662 to 1.64879, saving model to 1-roberta-0.h5
21984/21984 [==============================] - 290s 13ms/sample - loss: 1.5938 - activation_7_loss: 0.8213 - activation_8_loss: 0.7725 - val_loss: 1.6488 - val_activation_7_loss: 0.8488 - val_activation_8_loss: 0.8000
Epoch 3/3


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
test.head(3)

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative


In [11]:
test['selected_text'] = sel_text_test
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
2098,dabac40552,SUPER EXCITED. JUST BOUGHT MY TIX TO SEE HEY MONDAY ON ...,positive,super excited.
2986,5a17107aba,So i have done absolutely NOTHING all day today...how pi...,negative,pitiful
442,d39e6da95c,if you take backstreet boya and put them in silk mc ham...,neutral,if you take backstreet boya and put them in silk mc ham...
3510,bbfe533366,"In my moms hair salon, dying my hair",neutral,"in my moms hair salon, dying my hair"
970,51051cb758,"lunch break`s over, going back to work....",neutral,"lunch break`s over, going back to work...."
3294,7a1556637a,That`s alright. Add your egg and maybe some sort of le...,positive,that`s alright. add your egg and maybe some sort of lef...
2367,0f7696a955,Listening to maylene and wondering why cant I pull off a...,negative,wondering
251,005e2e60d3,Now I need to change my monitor,neutral,now i need to change my monitor
933,28db608c0d,Gonna go to work with my honey today!!!! So exciting!!!,positive,so exciting!!!
213,a172818c9e,misses my Mom today. She was my Best Friend and even tho...,positive,happy mother`s day
